In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
data_path_1 = '../../2020_MARCH (2).json'
data_path_2 = '../../2020_FEBRUARY (2).json'

In [ ]:
def location_history_processing_pipeline(raw_json_file_locations):
    raw_json = concat_src_files(raw_json_file_locations)
    places_df, location_df = extract_location_information_from_raw(raw_json)
    return location_df

In [ ]:
def concat_src_files(raw_json_file_locations):
    all_data = []
    for file in raw_json_file_locations:
        print(file)
        json_data = json.load(open(file, 'rb'))
        all_data += json_data['timelineObjects']
    return {'timelineObjects': all_data}

In [ ]:
def extract_location_information_from_raw(raw_json):
    places_data = []
    location_data = []

    for json_dict in raw_json['timelineObjects']:
        if "placeVisit" in json_dict:
            places_data.append(json_dict['placeVisit'])
            location_dict = json_dict['placeVisit']['location']
            location_dict['visit_start_time'] = json_dict['placeVisit']['duration']['startTimestampMs']
            location_dict['visit_end_time'] = json_dict['placeVisit']['duration']['endTimestampMs']
            location_data.append(location_dict)
    return pd.DataFrame(places_data), pd.DataFrame(location_data)

In [ ]:
df1 = location_history_processing_pipeline([data_path_1, data_path_2])

In [ ]:
df1[df1['name'] == 'Maxima XX'].visit_start_time.apply(lambda x: datetime.fromtimestamp(float(x) / 1000))


In [ ]:
cases = pd.read_csv("covid-cases-coordinates.csv")

In [ ]:
E7 = 10000000

#cases['Latitude'] = cases['Latitude'].apply(lambda x: int(x * E7))
#cases['Longitude'] = cases['Longitude'].apply(lambda x: int(x * E7))

In [ ]:
cases['Time'] = pd.to_datetime(cases['Time'])

In [ ]:
cases

# Risk heuristics

In [ ]:
DANGEROUS_ZONE_RANGE = 0.1
CRITICAL_OVERFLOW_TIMES = 5

def diminishing_importance(x_raw, x_limit, speed):
    x = (x_limit - min(x_limit, x_raw)) * 1.0 / x_limit
    #print(x)
    #if x >= 1 - DANGEROUS_ZONE_RANGE:
    #    return 1 - DANGEROUS_ZONE_RANGE + min(x_limit / x_raw * (DANGEROUS_ZONE_RANGE / CRITICAL_OVERFLOW_TIMES), DANGEROUS_ZONE_RANGE)
    
    return max(0, np.exp(speed * (x - 1)))

In [ ]:
!pip install geopy

In [ ]:
import geopy.distance
from datetime import datetime

TIMESTAMP_PREC = 1000.0
MAX_TIME_PASSED = 10000

def distance_between_points(latitude1, longitude1, latitude2, longitude2):
    return geopy.distance.geodesic((latitude1,longitude1),(latitude2,longitude2)).m

def is_in_date_range(date_x, date_start, date_end):
    return date_x >= date_start and date_x <= date_end

def get_top3_cases(place, cases):
    cases_extended = cases.copy()
    cases_extended['distance'] = cases_extended.apply(lambda x: distance_between_points(x.Latitude, x.Longitude, place.latitudeE7 * 1.0 / E7, place.longitudeE7 * 1.0 / E7), axis=1)
    cases_extended['time_passed'] = cases_extended.apply(lambda x: 0 if is_in_date_range(x.Time, datetime.fromtimestamp(float(place.visit_start_time) / TIMESTAMP_PREC), datetime.fromtimestamp(float(place.visit_end_time) / TIMESTAMP_PREC))\
                                                         else (datetime.fromtimestamp(int(place["visit_start_time"]) / TIMESTAMP_PREC) - x.Time).total_seconds() // 60, axis = 1)
    
    cases_extended['time_passed'] = cases_extended.apply(lambda x: MAX_TIME_PASSED if x['time_passed'] < 0 else x['time_passed'], axis = 1)
    
    return cases_extended.sort_values(by=["distance","time_passed"], ascending=True)[:3]

In [ ]:
get_top3_cases(df1.iloc[3], cases)

,No,Address,Name,City,Time,Latitude,Longitude,distance,time_passed
17,18,Liepojos pl. 1,Palangos oro uostas,Palanga,2020-03-09 00:10:00,55.971459,21.097289,426532.418401,10000.0
55,56,Liepojos pl. 1,Palangos oro uostas,Palanga,2020-03-16 15:25:00,55.971459,21.097289,426532.418401,10000.0
9,10,Venecijos akl. 2,Atostogų parkas,Žibininkai,2020-03-10 06:00:00,55.958187,21.124458,428587.692607,10000.0


In [ ]:
DURATION_START_LIMIT = 15
DURATION_END_LIMIT = 180

MAX_DISTANCE = 1000 # meters
MAX_TIME = 7200 # 15 days in minutes

TIME_DIMINISHING_SPEED = 10
DISTANCE_DIMINISHING_SPEED = 6

In [ ]:
def calculate_harmonic_mean(x, y):
    return 2 * x * y / (x + y)

def get_duration_multiplier(duration):
    return min(1 + max(duration - DURATION_START_LIMIT, 0) / DURATION_END_LIMIT, 2)

places_by_time = df1.sort_values(by="visit_start_time",ascending=False)

def duration_between_timestamps(timestamp1, timestamp2):
    return abs((datetime.fromtimestamp(float(timestamp2) / TIMESTAMP_PREC) - datetime.fromtimestamp(float(timestamp1) / TIMESTAMP_PREC)).seconds // 60)

components_values = []
for i in range(places_by_time.shape[0]):
    place = places_by_time.iloc[i]
    
    duration = duration_between_timestamps(place.visit_start_time, place.visit_end_time)
    
    duration_multiplier = get_duration_multiplier(duration)
    
    top3_cases = get_top3_cases(place, cases)
    
    print(top3_cases)
    for j in range(3):
        top_case = top3_cases.iloc[j]
        
        distance_factor = diminishing_importance(top_case.distance, MAX_DISTANCE, DISTANCE_DIMINISHING_SPEED)
        time_factor = diminishing_importance(top_case.time_passed, MAX_TIME, TIME_DIMINISHING_SPEED)
        
        print(place)
        print(top_case)
        print('--')
        
        score = (duration_multiplier / 2) * calculate_harmonic_mean(distance_factor, time_factor)
        
        components_values.append(top_case.values.tolist() + [place.address, duration, duration_multiplier, distance_factor, time_factor, place.visit_start_time, score])
        

components = pd.DataFrame(components_values, columns=["No", "Address", "Name", "City", "Time", "Latitude", "Longitude", "distance", "time_passed", "address", "duration","duration_multiplier","distance_factor","time_factor", "visit_start_time", "score"])
sorted_components = components.sort_values(by="score", ascending=False)

    No            Address     Name    City                Time   Latitude  \
0    1  S. Daukanto g. 23      NaN  Kaunas 2020-03-07 18:00:00  54.898015   
47  48     Veiverių g. 25     Aibe  Kaunas 2020-03-14 14:00:00  54.885751   
46  47     Veiverių g. 27  Camelia  Kaunas 2020-03-16 14:30:00  54.885465   

    Longitude     distance  time_passed  
0   23.913413  5619.355857      17123.0  
47  23.895724  6006.988377       7283.0  
46  23.895646  6032.368706       4373.0  
address               Gaižiūnų g. 5-2\nKaunas 50129\nLietuva
latitudeE7                                         549328346
locationConfidence                                   90.7399
longitudeE7                                        238499718
name                                             Aswo Baltic
placeId                          ChIJhUoYoG4f50YR8VT_9J1TiIQ
semanticType                                             NaN
sourceInfo                        {'deviceTag': -1914399071}
visit_end_time                     

Name: 0, dtype: object
--
address               Gaižiūnų g. 5-2\nKaunas 50129\nLietuva
latitudeE7                                         549328346
locationConfidence                                     89.22
longitudeE7                                        238499718
name                                             Aswo Baltic
placeId                          ChIJhUoYoG4f50YR8VT_9J1TiIQ
semanticType                                             NaN
sourceInfo                        {'deviceTag': -1914399071}
visit_end_time                                 1584535407459
visit_start_time                               1584445932840
Name: 70, dtype: object
No                              48
Address             Veiverių g. 25
Name                          Aibe
City                        Kaunas
Time           2020-03-14 14:00:00
Latitude                   54.8858
Longitude                  23.8957
distance                   6006.99
time_passed                   4312
Name: 47, dtype: object
-

    No           Address                 Name        City                Time  \
17  18    Liepojos pl. 1  Palangos oro uostas     Palanga 2020-03-09 00:10:00   
55  56    Liepojos pl. 1  Palangos oro uostas     Palanga 2020-03-16 15:25:00   
9   10  Venecijos akl. 2      Atostogų parkas  Žibininkai 2020-03-10 06:00:00   

     Latitude  Longitude       distance  time_passed  
17  55.971459  21.097289  425974.511124       9644.0  
55  55.971459  21.097289  425974.511124      10000.0  
9   55.958187  21.124458  428027.070420       7854.0  
address                191 40 Sollentuna\nSverige
latitudeE7                              594091861
locationConfidence                        97.1059
longitudeE7                             179622110
name                           Helenelund station
placeId               ChIJx_TCNpSeX0YRf4hC1AyXhjY
semanticType                                  NaN
sourceInfo             {'deviceTag': -1914399071}
visit_end_time                      1584284246876
visit

    No           Address                 Name        City                Time  \
17  18    Liepojos pl. 1  Palangos oro uostas     Palanga 2020-03-09 00:10:00   
55  56    Liepojos pl. 1  Palangos oro uostas     Palanga 2020-03-16 15:25:00   
9   10  Venecijos akl. 2      Atostogų parkas  Žibininkai 2020-03-10 06:00:00   

     Latitude  Longitude       distance  time_passed  
17  55.971459  21.097289  425726.140883       6733.0  
55  55.971459  21.097289  425726.140883      10000.0  
9   55.958187  21.124458  427781.250228       4943.0  
address               Hanstavägen 55 F\n164 53 Kista\nSverige
latitudeE7                                          594023097
locationConfidence                                     92.406
longitudeE7                                         179457338
name                                           Kista Galleria
placeId                           ChIJ3wMi6vKeX0YRrFmHpKBsHg0
semanticType                                              NaN
sourceInfo           

    No           Address                 Name        City                Time  \
17  18    Liepojos pl. 1  Palangos oro uostas     Palanga 2020-03-09 00:10:00   
55  56    Liepojos pl. 1  Palangos oro uostas     Palanga 2020-03-16 15:25:00   
9   10  Venecijos akl. 2      Atostogų parkas  Žibininkai 2020-03-10 06:00:00   

     Latitude  Longitude       distance  time_passed  
17  55.971459  21.097289  425876.727532       5108.0  
55  55.971459  21.097289  425876.727532      10000.0  
9   55.958187  21.124458  427931.150490       3318.0  
address               Isafjordsgatan 22\n164 40 Kista\nSverige
latitudeE7                                           594048322
locationConfidence                                     95.5533
longitudeE7                                          179494929
name                                                 KTH Kista
placeId                            ChIJwVNwM-2eX0YRQ79zxxBryak
semanticType                                         TYPE_WORK
sourceInfo    

...

    No           Address                 Name        City                Time  \
17  18    Liepojos pl. 1  Palangos oro uostas     Palanga 2020-03-09 00:10:00   
55  56    Liepojos pl. 1  Palangos oro uostas     Palanga 2020-03-16 15:25:00   
9   10  Venecijos akl. 2      Atostogų parkas  Žibininkai 2020-03-10 06:00:00   

     Latitude  Longitude       distance  time_passed  
17  55.971459  21.097289  421198.732015        10000  
55  55.971459  21.097289  421198.732015        10000  
9   55.958187  21.124458  423253.762432        10000  
address                     169 36 Solna\nSverige
latitudeE7                              593664392
locationConfidence                        96.6963
longitudeE7                             179831867
name                            Näckrosen (Solna)
placeId               ChIJJw3EytydX0YR3rFP-fX8wbE
semanticType                                  NaN
sourceInfo             {'deviceTag': -1914399071}
visit_end_time                      1581264999399
visit

9   55.958187  21.124458  428587.692607        10000  
address               Surtsögatan 3\n164 55 Kista\nSverige
latitudeE7                                       594084042
locationConfidence                                 98.8275
longitudeE7                                      179379559
name                                         Surtsögatan 3
placeId                        ChIJceIa5e6eX0YR2nuntatr2_s
semanticType                                     TYPE_HOME
sourceInfo                      {'deviceTag': -1914399071}
visit_end_time                               1581182675885
visit_start_time                             1581074849867
Name: 89, dtype: object
No                              18
Address             Liepojos pl. 1
Name           Palangos oro uostas
City                       Palanga
Time           2020-03-09 00:10:00
Latitude                   55.9715
Longitude                  21.0973
distance                    426532
time_passed                  10000
Name: 17, dtype:

address               Isafjordsgatan 22\n164 40 Kista\nSverige
latitudeE7                                           594048322
locationConfidence                                     98.9201
longitudeE7                                          179494929
name                                                 KTH Kista
placeId                            ChIJwVNwM-2eX0YRQ79zxxBryak
semanticType                                         TYPE_WORK
sourceInfo                          {'deviceTag': -1914399071}
visit_end_time                                   1581019598668
visit_start_time                                 1580998587077
Name: 85, dtype: object
No                              18
Address             Liepojos pl. 1
Name           Palangos oro uostas
City                       Palanga
Time           2020-03-09 00:10:00
Latitude                   55.9715
Longitude                  21.0973
distance                    425877
time_passed                  10000
Name: 17, dtype: object
--
addr

    No           Address                 Name        City                Time  \
17  18    Liepojos pl. 1  Palangos oro uostas     Palanga 2020-03-09 00:10:00   
55  56    Liepojos pl. 1  Palangos oro uostas     Palanga 2020-03-16 15:25:00   
9   10  Venecijos akl. 2      Atostogų parkas  Žibininkai 2020-03-10 06:00:00   

     Latitude  Longitude       distance  time_passed  
17  55.971459  21.097289  426532.418401        10000  
55  55.971459  21.097289  426532.418401        10000  
9   55.958187  21.124458  428587.692607        10000  
address               Surtsögatan 3\n164 55 Kista\nSverige
latitudeE7                                       594084042
locationConfidence                                 98.9355
longitudeE7                                      179379559
name                                         Surtsögatan 3
placeId                        ChIJceIa5e6eX0YR2nuntatr2_s
semanticType                                     TYPE_HOME
sourceInfo                      {'deviceTa

Name: 55, dtype: object
--
address               Surtsögatan 3\n164 55 Kista\nSverige
latitudeE7                                       594084042
locationConfidence                                 98.3884
longitudeE7                                      179379559
name                                         Surtsögatan 3
placeId                        ChIJceIa5e6eX0YR2nuntatr2_s
semanticType                                     TYPE_HOME
sourceInfo                      {'deviceTag': -1914399071}
visit_end_time                               1580730917718
visit_start_time                             1580580119590
Name: 76, dtype: object
No                              10
Address           Venecijos akl. 2
Name               Atostogų parkas
City                    Žibininkai
Time           2020-03-10 06:00:00
Latitude                   55.9582
Longitude                  21.1245
distance                    428588
time_passed                  10000
Name: 9, dtype: object
--
    No           A

In [ ]:
diminishing_importance(5 * 600, MAX_TIME, TIME_DIMINISHING_SPEED)

In [ ]:
diminishing_importance(550, MAX_DISTANCE, DISTANCE_DIMINISHING_SPEED)

In [ ]:
W1 = 0.7
W2 = 0.2
W3 = 0.1

risk = W1 * sorted_components.iloc[0].score + W2 * sorted_components.iloc[1].score + W3 * sorted_components.iloc[2].score

risk

In [ ]:
sorted_components

,No,Address,Name,City,Time,Latitude,Longitude,distance,time_passed,address,duration,duration_multiplier,distance_factor,time_factor,visit_start_time,score
189,60,Geležinkelio g. 16,Vilniaus Geležinkelio stotis,Vilnius,2020-03-04 06:00:00,54.670220,25.284368,54.709632,338.0,Geležinkelio g. 16\nVilnius 02100\nLietuva,50,1.194444,0.720177,0.625350,1583314733400,0.399794
61,21,Sodų g. 22,Vilniaus stotis,Vilnius,2020-03-11 14:50:00,54.670763,25.281826,163.852125,182.0,Geležinkelio g. 16\nVilnius 02100\nLietuva,31,1.088889,0.374145,0.776640,1583941939236,0.274947
62,6,Sodų g. 22,Vilniaus stotis,Vilnius,2020-03-10 17:00:00,54.670763,25.281826,163.852125,1492.0,Geležinkelio g. 16\nVilnius 02100\nLietuva,31,1.088889,0.374145,0.125906,1583941939236,0.102578
99,16,Karaliaus Mindaugo pr. 49,Kauno Akropolis,Kaunas,2020-03-07 06:00:00,54.890501,23.919621,37.018407,1867.0,Karaliaus Mindaugo pr. 49\nKaunas 44333\nLietuva,8,1.000000,0.800827,0.074791,1583665655021,0.068403
63,5,Vytauto pr. 24,Kauno stotis,Kaunas,2020-03-10 16:00:00,54.889049,23.927838,370.841215,1428.0,Kaunas 44362\nLietuva,25,1.055556,0.108062,0.137610,1583934509763,0.063892
20,47,Veiverių g. 27,Camelia,Kaunas,2020-03-16 14:30:00,54.885465,23.895646,6032.368706,0.0,Gaižiūnų g. 5-2\nKaunas 50129\nLietuva,850,2.000000,0.002479,1.000000,1584305437409,0.004945
105,1,S. Daukanto g. 23,NaN,Kaunas,2020-03-07 18:00:00,54.898015,23.913413,5619.355857,0.0,Gaižiūnų g. 5-2\nKaunas 50129\nLietuva,1372,2.000000,0.002479,1.000000,1583580858974,0.004945
69,5,Vytauto pr. 24,Kauno stotis,Kaunas,2020-03-10 16:00:00,54.889049,23.927838,1696.420985,49.0,Vydūno al. 25B\nKaunas 50288\nLietuva,427,2.000000,0.002479,0.934209,1583851745633,0.004944
87,1,S. Daukanto g. 23,NaN,Kaunas,2020-03-07 18:00:00,54.898015,23.913413,5619.355857,1506.0,Gaižiūnų g. 5-2\nKaunas 50129\nLietuva,1366,2.000000,0.002479,0.123481,1583687219881,0.004860
19,48,Veiverių g. 25,Aibe,Kaunas,2020-03-14 14:00:00,54.885751,23.895724,6006.988377,1970.0,Gaižiūnų g. 5-2\nKaunas 50129\nLietuva,850,2.000000,0.002479,0.064822,1584305437409,0.004775


In [ ]:
(datetime.fromtimestamp(int(sorted_components.iloc[0].visit_start_time)  / 1000) - sorted_components.iloc[0].Time).total_seconds() // 60#.seconds // 60

In [ ]:
datetime.fromtimestamp(int(sorted_components.iloc[0].visit_start_time)  / 1000)

In [ ]:
datetime.fromtimestamp(int(1584113900482) / 1000)

In [ ]:
sorted_components = sorted_components[["Address", "Name", "City", "Time", "distance", "time_passed", "address", "score"]]

In [ ]:
sorted_components = sorted_components.rename(columns={"Address": "Case Address", "address": "Your visit"})

In [ ]:
sorted_components[:3]

,Case Address,Name,City,Time,distance,time_passed,Your visit,score
120,S. Daukanto g. 23,NaN,Kaunas,2020-03-07 18:00:00,105.738799,339,5th floor\nLaisvės al. 59\nKaunas 44305\nLietuva,0.981344
189,Geležinkelio g. 16,Vilniaus Geležinkelio stotis,Vilnius,2020-03-04 06:00:00,54.709632,338,Geležinkelio g. 16\nVilnius 02100\nLietuva,0.799587
153,Karaliaus Mindaugo pr. 49,Kauno Akropolis,Kaunas,2020-03-07 06:00:00,37.018407,602,Karaliaus Mindaugo pr. 49\nKaunas 44333\nLietuva,0.762386


In [ ]:
df1.shape

In [ ]:
185 * 3